In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-04-26 02:25:29--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-04-26 02:25:29--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?jY-ri-Wid2mrOEpe1Ho01Cd3mkwnR3UeJLloVMDzqyDBBfpN_o_1UiBuQGDKspirjXiYjjJVdHxp98c47NMM0ZpHt8BT43h7qlulz1bFzIlNv4X9o

In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8ci8_aaa
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8ci8_aaa
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=c032b09c6b47de5bd57bd37c5cf8ec838bee96c85e219646b0edda7712bb74fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-ud72y48b/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [5]:
#load extension into environment
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [9]:
%%cu

#include<iostream>
#include<cstdio>
#include<cstdlib>
#include<cuda_runtime.h>
#include<iomanip>
using namespace std;

__global__ void minimum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] < input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void maximum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] > input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void sum(int *input) {
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

__global__ void mean_diff_sq(float *input, float mean) {
    input[threadIdx.x] -= mean;
    input[threadIdx.x] *= input[threadIdx.x];
}

__global__ void sum_floats(float *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

void copy_int_to_float(float *dest, int *src, int size){
    for(int i=0; i<size; i++)
        dest[i] = float(src[i]);
}

int main() {
    int n=1000;
    srand(n);
	  int *arr=new int[n];
    int min=20000;
    //# Generate Input array using rand()
	  for(int i=0;i<n;i++)
	  {
		  arr[i]=rand()%20000;
      if(arr[i]<min)
        min=arr[i];
      cout<<arr[i]<<" ";
	  }
    cout<<endl;
    int size = n*sizeof(int); //calculate no. of bytes for array
    int *arr_d, result;
    
    cudaMalloc((void **)&arr_d, size);
    
    //MIN
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    minimum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The minimum element is "<<result<<endl;
      
       
    //MAX
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    maximum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The maximum element is "<<result<<endl;
    
    //SUM
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    sum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The sum is "<<result<<endl;
    
    //AVERAGE
    
    float mean = float(result)/n;
    cout<<"The mean is "<<mean<<endl;
    
    //STANDARD DEVIATION
    float *arr_float;
    float *arr_std, stdValue;
    
    arr_float = (float *)malloc(n*sizeof(float));
    cudaMalloc((void **)&arr_std, n*sizeof(float));
    
    copy_int_to_float(arr_float, arr, n);
    
    cudaMemcpy(arr_std, arr_float, n*sizeof(float), cudaMemcpyHostToDevice);
    
    mean_diff_sq <<<1,n>>>(arr_std, mean);
    sum_floats<<<1,n/2>>>(arr_std);
    
    cudaMemcpy(&stdValue, arr_std, sizeof(float), cudaMemcpyDeviceToHost);
    
    
    stdValue = stdValue / n;
    std::cout << std::fixed << std::setprecision(2)<<"The variance is "<< stdValue << std::endl;
    stdValue = sqrt(stdValue);
    
    cout<<"The standard deviation is "<<stdValue<<endl;
    
    cudaFree(arr_d);
           
    return 0;
}

790 10779 13513 6398 3604 15141 11803 11570 6268 9855 13138 17283 16994 1147 16433 9553 15216 19500 1691 3952 4237 2517 4171 5346 6716 11718 16847 9840 12654 9836 12056 13444 616 1921 19842 572 13414 7997 8494 19682 14204 17985 16966 7550 19132 13399 17104 10700 9252 15147 11004 9841 17665 11527 11540 733 3245 8387 10573 15899 18223 18982 5695 15191 903 1889 12116 14318 6238 610 14000 16794 18595 7318 4344 14079 17070 17800 1131 2674 12948 12135 12515 6965 3662 4055 7698 3259 8794 14623 19158 3370 13605 1205 14913 10861 19446 7029 5179 2036 7640 15531 18830 2587 19202 19527 13019 12624 17327 14150 15298 10275 6286 4165 13592 6300 4573 17642 9560 9719 12266 5070 13089 2223 2628 4355 13084 2074 11384 14615 463 15376 6499 15645 14316 5701 15172 7335 14677 8852 1485 6327 15479 4123 10492 9072 10424 11417 6714 19984 1137 15332 1406 14226 17556 4034 18581 6992 2461 6318 1608 2924 18046 4459 18569 12362 6512 10094 19697 1189 18946 1183 7516 14425 5306 14360 19849 12082 5778 6564 8418 6915 182

In [10]:
%%writefile code.cpp
#include <stdio.h>
#include <algorithm> 
#include <chrono> 
#include <omp.h>
#include <vector>
#include <cmath>
#include <iostream> 
using namespace std; 
using namespace std::chrono; 
int main()
{
    int l = 1e6;
    vector<int> arr(l);

    for(int i=0;i<l;i++){
        arr[i] = (rand()%10001);
    }
    
    long i, max_val=0, min_val=0;
    double Val = 0, std_dev=0;
    long timeSerial = 0, timeParallel = 0;
    auto start = high_resolution_clock::now(); 
        
    cout<<"\n\n###########  Max in Array ###############\n\n";
    
    for( i=0;i<l; i++)
    {
        if(arr[i] > max_val)
        {
            max_val = arr[i];  
        }
    }
  
    auto stop = high_resolution_clock::now(); 
    auto duration = duration_cast<microseconds>(stop - start); 

    printf("\nmax_val = %ld\n", max_val);
    cout << "Time taken for max in array Serial : " << duration.count() << " microseconds" << endl; 
    timeSerial+= duration.count();
    
    
    omp_set_num_threads(4);
    int cur_max = 0;
    start = high_resolution_clock::now(); 

    #pragma omp parallel for
	for (i = 0; i < l; i = i + 1) {
		if (arr[i] > cur_max)
        #pragma omp critical
			if (arr[i] > cur_max)
				cur_max = arr[i];
    }
    stop = high_resolution_clock::now(); 
    duration = duration_cast<microseconds>(stop - start); 

    printf("\nmax_val = %d\n", cur_max);
    cout << "Time taken for max in array in Parallel : " << duration.count() << " microseconds" << endl; 
    timeParallel+= duration.count();



    start = high_resolution_clock::now(); 
    cout<<"\n\n###########  Min in Array ###############\n";
   
    for( i=0;i<l; i++)
    {
        if(arr[i] < min_val)
        {
            min_val = arr[i];  
        }
    }
  
    stop = high_resolution_clock::now(); 
    duration = duration_cast<microseconds>(stop - start); 

    printf("\nmin_val = %ld\n", min_val);
    cout << "Time taken for min in array : " << duration.count() << " microseconds" << endl; 
    timeSerial+= duration.count();

    int cur_min = 0;
    start = high_resolution_clock::now(); 

   #pragma omp parallel for
	for (i = 0; i < l; i = i + 1) {
		if (arr[i] < cur_min)
        #pragma omp critical
			if (arr[i] < cur_min)
				cur_min = arr[i];
    }
    stop = high_resolution_clock::now(); 
    duration = duration_cast<microseconds>(stop - start); 

    printf("\nmin_val = %d\n", cur_min);
    cout << "Time taken for min in array in Parallel : " << duration.count() << " microseconds" << endl; 
    timeParallel+= duration.count();

/////////////// MEAN /////////////////

    start = high_resolution_clock::now(); 
    cout<<"\n\n###########  Mean of elements in Array ###############\n";
   
    for( i = 0; i <l; i++) {
        Val = Val+ arr[i];
    }
    Val/= l;
    stop = high_resolution_clock::now(); 
    duration = duration_cast<microseconds>(stop - start); 
    printf("\nMean Value = %f\n", Val);
    cout << "Time taken for mean of elements in Serial : " << duration.count() << " microseconds" << endl; 
    timeSerial+= duration.count();


    Val=0;
    start = high_resolution_clock::now(); 
    #pragma omp parallel for reduction(+:Val)
    for( i = 0; i <l; i++) {
        Val = Val+ arr[i];
    }
    Val/= l;
    stop = high_resolution_clock::now(); 
    duration = duration_cast<microseconds>(stop - start); 
    printf("\n\nMean Value = %f\n", Val);
    cout << "Time taken for mean of elements in Parallel : " << duration.count() << " microseconds" << endl; 
    timeParallel+= duration.count();



    start = high_resolution_clock::now(); 
    cout<<"\n\n###########  Standard Deviation of elements in Array ###############\n";
  
    for( i = 0; i <l; i++) {
        std_dev = std_dev + pow(arr[i] - Val, 2  );
    }
    std_dev/= l;
    std_dev = sqrt(std_dev);
    stop = high_resolution_clock::now(); 
    duration = duration_cast<microseconds>(stop - start); 
    printf("\nValue = %f\n", std_dev);
    cout << "Time taken for standard deviation of elements in Serial: " << duration.count() << " microseconds" << endl; 
    timeSerial+= duration.count();

    start = high_resolution_clock::now(); 
    
    #pragma omp parallel for reduction(+:std_dev)
    for( i = 0; i <l; i++) {
        std_dev = std_dev + pow(arr[i] - Val, 2  );
    }
    std_dev/= l;
    std_dev = sqrt(std_dev);
    stop = high_resolution_clock::now(); 
    duration = duration_cast<microseconds>(stop - start); 
    printf("\n\nValue = %f\n", std_dev);
    cout << "Time taken for standard deviation of elements in Parallel : " << duration.count() << " microseconds" << endl; 
    timeParallel+= duration.count();
    

    cout<<"\nTotal time in Serial : "<<timeSerial<<endl;
    
    cout<<"\nTotal time in Parallel : "<<timeParallel<<endl;
    
    return 0;
}

Overwriting code.cpp


In [11]:
%%script bash
g++ -fopenmp code.cpp -o code
./code



###########  Max in Array ###############


max_val = 10000
Time taken for max in array Serial : 2706 microseconds

max_val = 10000
Time taken for max in array in Parallel : 3474 microseconds


###########  Min in Array ###############

min_val = 0
Time taken for min in array : 2834 microseconds

min_val = 0
Time taken for min in array in Parallel : 3132 microseconds


###########  Mean of elements in Array ###############

Mean Value = 4995.665438
Time taken for mean of elements in Serial : 4664 microseconds


Mean Value = 4995.665438
Time taken for mean of elements in Parallel : 3180 microseconds


###########  Standard Deviation of elements in Array ###############

Value = 2885.908529
Time taken for standard deviation of elements in Serial: 14404 microseconds


Value = 2885.908529
Time taken for standard deviation of elements in Parallel : 12390 microseconds

Total time in Serial : 24608

Total time in Parallel : 22176
